In [ ]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


SEARCH_WORD = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

spreadSheet_ids = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "アスカ狭山店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    }

# 検索キーワードよりホール名取得
SPREADSHEET_ID = spreadSheet_ids[SEARCH_WORD]

# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)
spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + SEARCH_WORD + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{SEARCH_WORD}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{SEARCH_WORD}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"], cherry=True).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
df["area"] = df["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = df["model_name"].unique()
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 58368 x 17
以下の日付のデータが含まれています
2025-04-22 00:00:00
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 8: SミスタージャグラーKK, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-04-22,ゴーゴージャグラー3,1001,2168,7,6,-191,309.7,361.3,5.62,166.8,2025-04,22,1,1,a
1,EXA FIRST,2025-04-22,ゴーゴージャグラー3,1002,5061,22,19,947,230.0,266.4,5.99,123.4,2025-04,22,1,2,a
2,EXA FIRST,2025-04-22,ゴーゴージャグラー3,1003,246,1,0,-50,246.0,0.0,5.84,246.0,2025-04,22,1,3,a
3,EXA FIRST,2025-04-22,ゴーゴージャグラー3,1004,3534,20,7,994,176.7,504.9,6.22,130.9,2025-04,22,1,4,a
4,EXA FIRST,2025-04-22,ゴーゴージャグラー3,1005,2646,9,8,-350,294.0,330.8,6.09,155.6,2025-04,22,1,5,a


## 過去7日間の差枚と翌日の結果を比較

In [ ]:
# model_name = model_list[1]
# today = datetime.date.today()
# target_date = today - datetime.timedelta(days=1)
# start = today - relativedelta(days=1)
# end = today - relativedelta(days=30)
# unit_no = 1021

# df_tmp = df.copy()
# df_tmp = df_tmp[(df_tmp["model_name"] == model_name)]
# df_tmp = df_tmp[(df_tmp["date"].dt.date <= start) & (df_tmp["date"].dt.date >= end)]
# df_tmp = df_tmp[(df_tmp["unit_no"] >= unit_no)]

# medals = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="medals", aggfunc="sum",)
# game = df_tmp.pivot_table(index=["model_name", "unit_no", ], columns="date", values="game", aggfunc="sum",)
# bb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="BB", aggfunc="sum",)
# rb = df_tmp.pivot_table(index=["model_name", "unit_no"], columns="date", values="RB", aggfunc="sum",)
# rb_rate = (game / rb).round(1)
# total_rate = (game / (bb+rb)).round(1)

# medals = medals.iloc[:, 7:].iloc[:, ::-1]
# rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]
# total_rate = total_rate.iloc[:, 7:].iloc[:, ::-1]

# rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
# rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
# # rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]

# rolling_7d_rank = rolling_7d_sum.rank(method="min", ascending=True).astype(int)
# rolling_7d_rank.columns = [c.replace("sum", "rank") for c in rolling_7d_rank.columns]

# medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
# rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
# total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
# rolling_7d_sum.columns = pd.MultiIndex.from_product([["7D_sum"], rolling_7d_sum.columns])
# rolling_7d_rank.columns = pd.MultiIndex.from_product([["RANK"], rolling_7d_rank.columns])

# print(rolling_7d_sum.shape)
# print(rolling_7d_rank.shape)
# print(rb_rate.shape)
# print(total_rate.shape)
# display(rolling_7d_sum.head(2))
# display(rolling_7d_rank.head(2))
# display(rb_rate.head(2))
# display(total_rate.head(2))


(61, 17)
(61, 17)
(61, 23)
(61, 23)


C:\Users\mkyag\AppData\Local\Temp\ipykernel_29416\2245148149.py:24: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]


7D_sum                                  \
                   25-04-22_7d_sum 25-04-21_7d_sum 25-04-20_7d_sum   
model_name unit_no                                                   
マイジャグラーV   1021            -1225.0           940.0          -802.0   
           1022             1123.0           135.0          2035.0   

                                                                    \
                   25-04-19_7d_sum 25-04-18_7d_sum 25-04-17_7d_sum   
model_name unit_no                                                   
マイジャグラーV   1021             -672.0         -1245.0         -2389.0   
           1022             1170.0          2038.0          2574.0   

                                                                    \
                   25-04-16_7d_sum 25-04-15_7d_sum 25-04-14_7d_sum   
model_name unit_no                                                   
マイジャグラーV   1021            -1068.0         -1333.0         -3765.0   
           1022             2248.0          2004.0          1406.0   

                                                                    \
                   25-04-13_7d_sum 25-04-12_7d_sum 25-04-11_7d_sum   
model_name unit_no                                                   
マイジャグラーV   1021            -1129.0          -667.0            24.0   
           1022             1583.0           904.0           118.0   

                                                                    \
                   25-04-10_7d_sum 25-04-09_7d_sum 25-04-08_7d_sum   
model_name unit_no                                                   
マイジャグラーV   1021             1016.0           528.0           -42.0   
           1022              827.0           262.0          -474.0   

                                                    
                   25-04-07_7d_sum 25-04-06_7d_sum  
model_name unit_no                                  
マイジャグラーV   1021              381.0         -1469.0  
           1022             -356.0         -2633.0

RANK                                    \
                   25-04-22_7d_rank 25-04-21_7d_rank 25-04-20_7d_rank   
model_name unit_no                                                      
マイジャグラーV   1021                  20               45               29   
           1022                  47               35               51   

                                                                       \
                   25-04-19_7d_rank 25-04-18_7d_rank 25-04-17_7d_rank   
model_name unit_no                                                      
マイジャグラーV   1021                  27               24               14   
           1022                  50               52               53   

                                                                       \
                   25-04-16_7d_rank 25-04-15_7d_rank 25-04-14_7d_rank   
model_name unit_no                                                      
マイジャグラーV   1021                  19               20                3   
           1022                  51               49               48   

                                                                       \
                   25-04-13_7d_rank 25-04-12_7d_rank 25-04-11_7d_rank   
model_name unit_no                                                      
マイジャグラーV   1021                  23               22               25   
           1022                  45               41               27   

                                                                       \
                   25-04-10_7d_rank 25-04-09_7d_rank 25-04-08_7d_rank   
model_name unit_no                                                      
マイジャグラーV   1021                  48               39               31   
           1022                  43               34               26   

                                                      
                   25-04-07_7d_rank 25-04-06_7d_rank  
model_name unit_no                                    
マイジャグラーV   1021                  32               17  
           1022                  29               12

RB_RATE                                              \
date               2025-04-22 2025-04-21 2025-04-20 2025-04-19 2025-04-18   
model_name unit_no                                                          
マイジャグラーV   1021         917.0      405.9     1240.5      348.1      269.1   
           1022         418.9      440.0      344.9      698.2      638.0   

                                                                           \
date               2025-04-17 2025-04-16 2025-04-15 2025-04-14 2025-04-13   
model_name unit_no                                                          
マイジャグラーV   1021         459.3      434.9      251.1      940.8      659.8   
           1022         484.6      360.9      326.5      306.4      613.0   

                    ...                                              \
date                ... 2025-04-09 2025-04-08 2025-04-07 2025-04-06   
model_name unit_no  ...                                               
マイジャグラーV   1021     ...      435.4      405.0      253.3      333.8   
           1022     ...      349.8        inf      261.3      468.0   

                                                                           \
date               2025-04-05 2025-04-04 2025-04-03 2025-04-02 2025-04-01   
model_name unit_no                                                          
マイジャグラーV   1021         429.1      352.6      409.1      326.9      653.3   
           1022         261.6      595.2      508.7      362.1      488.2   

                               
date               2025-03-31  
model_name unit_no             
マイジャグラーV   1021         409.1  
           1022         508.7  

[2 rows x 23 columns]

TOTAL_RATE                                              \
date               2025-04-22 2025-04-21 2025-04-20 2025-04-19 2025-04-18   
model_name unit_no                                                          
マイジャグラーV   1021         305.7      142.0      275.7      160.7      134.5   
           1022         142.8      180.0      145.2      161.1      638.0   

                                                                           \
date               2025-04-17 2025-04-16 2025-04-15 2025-04-14 2025-04-13   
model_name unit_no                                                          
マイジャグラーV   1021         212.0      169.1      127.3      209.1      208.4   
           1022         150.4      142.4      146.9      126.6      245.2   

                    ...                                              \
date                ... 2025-04-09 2025-04-08 2025-04-07 2025-04-06   
model_name unit_no  ...                                               
マイジャグラーV   1021     ...      163.3      186.9      104.8      161.5   
           1022     ...      145.8      578.0      120.6      234.0   

                                                                           \
date               2025-04-05 2025-04-04 2025-04-03 2025-04-02 2025-04-01   
model_name unit_no                                                          
マイジャグラーV   1021         158.1      146.9      157.3      208.0      186.7   
           1022         137.2      163.7      254.3      181.1      167.4   

                               
date               2025-03-31  
model_name unit_no             
マイジャグラーV   1021         157.3  
           1022         254.3  

[2 rows x 23 columns]

In [ ]:
# # 同じ列数を前提（もしくは短い方に合わせる）
# columns_interleaved = [
#     col for pair in zip(rolling_7d_rank.columns, rolling_7d_sum.columns, medals.columns, rb_rate.columns, total_rate.columns) for col in pair
# ]
# merged = pd.concat([rolling_7d_rank, rolling_7d_sum, medals, rb_rate, total_rate], axis=1)[columns_interleaved]
# # 空行を追加
# empty_index = pd.MultiIndex.from_tuples([("", "")], names=merged.index.names)
# merged_with_blanks = pd.DataFrame(
#     [[""] * merged.shape[1]], columns=merged.columns, index=empty_index
# )
# empty_row = merged_with_blanks.copy()
# blocks = [
#     merged.iloc[0:6],
#     merged.iloc[6:12],
#     merged.iloc[12:21],
#     merged.iloc[21:30],
#     merged.iloc[30:46],
#     merged.iloc[46:54],
#     merged.iloc[54:61],
# ]
# for block in blocks:
#     merged_with_blanks = pd.concat([merged_with_blanks, block, empty_row])

# # rank = rolling_7d_sum.iloc[:, 1].rank(method="min", ascending=True).astype(int)
# merged_with_blanks.to_csv(f"{model_name}_merged.csv", encoding="utf_8_sig")
# merged_with_blanks.head()

RANK          7D_sum              MEDALS  \
                   25-04-22_7d_rank 25-04-22_7d_sum 2025-04-22 00:00:00   
model_name unit_no                                                        
                                                                          
マイジャグラーV   1021                  20         -1225.0                -374   
           1022                  47          1123.0                1112   
           1023                  37          -116.0                -162   
           1024                  48          1149.0                 479   

                               RB_RATE          TOTAL_RATE             RANK  \
                   2025-04-22 00:00:00 2025-04-22 00:00:00 25-04-21_7d_rank   
model_name unit_no                                                            
                                                                              
マイジャグラーV   1021                  917.0               305.7               45   
           1022                  418.9               142.8               35   
           1023                  301.8               156.5               17   
           1024                  407.2               135.8               37   

                            7D_sum              MEDALS             RB_RATE  \
                   25-04-21_7d_sum 2025-04-21 00:00:00 2025-04-21 00:00:00   
model_name unit_no                                                           
                                                                             
マイジャグラーV   1021              940.0                 724               405.9   
           1022              135.0                -462               440.0   
           1023            -1419.0                -762               467.2   
           1024              202.0                2515               309.9   

                            TOTAL_RATE  ...             RANK          7D_sum  \
                   2025-04-21 00:00:00  ... 25-04-07_7d_rank 25-04-07_7d_sum   
model_name unit_no                      ...                                    
                                        ...                                    
マイジャグラーV   1021                  142.0  ...               32           381.0   
           1022                  180.0  ...               29          -356.0   
           1023                  233.6  ...               22         -1206.0   
           1024                  121.9  ...               14         -2200.0   

                                MEDALS             RB_RATE  \
                   2025-04-07 00:00:00 2025-04-07 00:00:00   
model_name unit_no                                           
                                                             
マイジャグラーV   1021                   1618               253.3   
           1022                   1615               261.3   
           1023                   -685               336.0   
           1024                    391               309.9   

                            TOTAL_RATE             RANK          7D_sum  \
                   2025-04-07 00:00:00 25-04-06_7d_rank 25-04-06_7d_sum   
model_name unit_no                                                        
                                                                          
マイジャグラーV   1021                  104.8               17         -1469.0   
           1022                  120.6               12         -2633.0   
           1023                  252.0               16         -1774.0   
           1024                  146.8               13         -2573.0   

                                MEDALS             RB_RATE          TOTAL_RATE  
                   2025-04-06 00:00:00 2025-04-06 00:00:00 2025-04-06 00:00:00  
model_name unit_no                                                              
                                                                                
マイジャグラーV   1021                   -579               333.8               161.5  
           1022             

## コード確認テスト

In [ ]:
def extract_and_merge_model_data(df, model_name, start_date, end_date):
    # 対象期間・モデルのデータを抽出
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model_name)]
    
    # 各種ピボットテーブル
    medals = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="medals", aggfunc="sum")
    game = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="game", aggfunc="sum",)
    rb_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="RB_rate", aggfunc="sum")
    total_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Total_rate", aggfunc="sum")
    grape_rate = df_filtered.pivot_table(index=["area", "model_name", "unit_no"], columns="date", values="Grape_rate", aggfunc="sum")
    
    # 日付列を反転・スライス
    medals = medals.iloc[:, 7:].iloc[:, ::-1]
    game = game.iloc[:, 7:].iloc[:, ::-1]
    rb_rate = rb_rate.iloc[:, 7:].iloc[:, ::-1]
    total_rate = total_rate.iloc[:, 7:].iloc[:, ::-1]
    grape_rate = grape_rate.iloc[:, 7:].iloc[:, ::-1]

    # 7日間累積とランク
    rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
    rolling_7d_sum.columns = [f"{col.strftime('%y-%m-%d')}_7d_sum" for col in rolling_7d_sum.columns]
    rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    rolling_7d_rank = rolling_7d_sum.rank(method="min", ascending=True).fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling_7d_rank.columns = [c.replace("sum", "rank") for c in rolling_7d_rank.columns]
    rolling_7d_sum = rolling_7d_sum.iloc[:, ::-1]
    rolling_7d_rank = rolling_7d_rank.iloc[:, ::-1]

    # MultiIndex化（ラベル付け）
    medals.columns = pd.MultiIndex.from_product([["MEDALS"], medals.columns])
    game.columns = pd.MultiIndex.from_product([["GAME"], game.columns])
    rb_rate.columns = pd.MultiIndex.from_product([["RB_RATE"], rb_rate.columns])
    total_rate.columns = pd.MultiIndex.from_product([["TOTAL_RATE"], total_rate.columns])
    grape_rate.columns = pd.MultiIndex.from_product([["GRAPE_RATE"], grape_rate.columns])
    rolling_7d_sum.columns = pd.MultiIndex.from_product([["7D_sum"], rolling_7d_sum.columns])
    rolling_7d_rank.columns = pd.MultiIndex.from_product([["RANK"], rolling_7d_rank.columns])

    # 列を交互に整列して統合・NaN除去
    interleaved_cols  = [
        col for pair in zip(
            rolling_7d_rank.columns,
            rolling_7d_sum.columns,
            medals.columns,
            game.columns,
            rb_rate.columns,
            total_rate.columns,
            grape_rate.columns
            ) for col in pair
        ]
    merged = pd.concat([
        rolling_7d_rank, rolling_7d_sum, medals, game, rb_rate, total_rate, grape_rate], 
                       axis=1
        )[interleaved_cols]
    merged = merged[~merged.iloc[:, 2].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples([("", " ", " ")], names=merged.index.names)
            empty_row = pd.DataFrame([[""] * area_merged.shape[1]], index=empty_index, columns=area_merged.columns)
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])
    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    return merged_by_area


today = datetime.date.today()
start_date = today - relativedelta(days=1)
end_date = start_date - relativedelta(days=30)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' ]
for model in model_list:
    merged_by_area = extract_and_merge_model_data(df, model, start_date, end_date)
    # モデル間の区切り用空行追加して結合
    empty_index = pd.MultiIndex.from_tuples([(" ", " ")], names=merged_by_area.index.names)
    empty_row = pd.DataFrame([[""] * merged_by_area.shape[1]], index=empty_index, columns=merged_by_area.columns)
    merged_by_model = pd.concat([merged_by_model, merged_by_area, empty_row], axis=0)
# 出力
merged_by_model.to_csv(f"merged_by_model_{start_date}_{end_date}.csv", encoding="utf_8_sig")
display(merged_by_model)

対象日付: 2025-04-23


C:\Users\mkyag\AppData\Local\Temp\ipykernel_29416\1371020072.py:22: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
C:\Users\mkyag\AppData\Local\Temp\ipykernel_29416\1371020072.py:22: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
C:\Users\mkyag\AppData\Local\Temp\ipykernel_29416\1371020072.py:22: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  rolling_7d_sum = medals.iloc[:, ::-1].rolling(window=7, axis=1, min_periods=1).sum().iloc[:, ::-1].iloc[:, :-6]
C:\Users\mkyag\AppData\Local\Te

RANK          7D_sum              MEDALS  \
                   25-04-22_7d_rank 25-04-22_7d_sum 2025-04-22 00:00:00   
model_name unit_no                                                        
マイジャグラーV   1021                  20         -1225.0              -374.0   
           1022                  47          1123.0              1112.0   
           1023                  37          -116.0              -162.0   
           1024                  48          1149.0               479.0   
           1025                  50          1506.0              -109.0   
...                             ...             ...                 ...   
ミスタージャグラー  1111                   8          -353.0               438.0   
           1112                  14          2577.0                 9.0   
           1113                  11          1488.0              -674.0   
                                                                          
                                                                          

                                  GAME             RB_RATE  \
                   2025-04-22 00:00:00 2025-04-22 00:00:00   
model_name unit_no                                           
マイジャグラーV   1021                  917.0               917.0   
           1022                 6283.0               418.9   
           1023                 4225.0               301.8   
           1024                 1629.0               407.2   
           1025                 2006.0               501.5   
...                                ...                 ...   
ミスタージャグラー  1111                 1808.0               452.0   
           1112                 2584.0               516.8   
           1113                 2321.0               464.2   
                                                             
                                                             

                            TOTAL_RATE          GRAPE_RATE             RANK  \
                   2025-04-22 00:00:00 2025-04-22 00:00:00 25-04-21_7d_rank   
model_name unit_no                                                            
マイジャグラーV   1021                  305.7                5.29               45   
           1022                  142.8                5.75               35   
           1023                  156.5                5.49               17   
           1024                  135.8                5.59               37   
           1025                  182.4                5.45               32   
...                                ...                 ...              ...   
ミスタージャグラー  1111                  139.1                5.77                3   
           1112                  161.5                5.94               13   
           1113                  193.4                6.02               12   
                                                                              
                                                                              

                            7D_sum              MEDALS  ...  \
                   25-04-21_7d_sum 2025-04-21 00:00:00  ...   
model_name unit_no                                      ...   
マイジャグラーV   1021              940.0               724.0  ...   
           1022              135.0              -462.0  ...   
           1023            -1419.0              -762.0  ...   
           1024              202.0              2515.0  ...   
           1025                9.0              -356.0  ...   
...                            ...                 ...  ...   
ミスタージャグラー  1111             -791.0              -791.0  ...   
           1112             2568.0              2568.0  ...   
           1113             2162.0              2162.0  ...   
                                                        ...   
                                                        ...   

                               RB_RATE          TOTAL_RATE  \
                   2025-04-06 00:00:00 2025-04-06 00:00:00 

## 2024年のGWの分析

In [406]:
# model_name = model_list[1]
today = datetime.date.today()
target_date = today - datetime.timedelta(days=1)
start_date = datetime.date(2024, 5, 5)
end_date = datetime.date(2024, 4, 27)

merged_by_model = pd.DataFrame()
model_list = ['マイジャグラーV', 'アイムジャグラーEX-TP', 'ゴーゴージャグラー3', 'ファンキージャグラー2', 'ミスタージャグラー' ]
for model in model_list:
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered["date"].dt.date <= start_date) & (df_filtered["date"].dt.date >= end_date)]
    df_filtered = df_filtered[(df_filtered["model_name"] == model)]